In [233]:
import pandas as pd
import os
import json
import glob
import nltk
import sklearn
import numpy as np
import re

In [6]:
metadata_csv_path = os.path.join(os.path.abspath(os.path.curdir), "docs\\CORD-19-research-challenge\\metadata.csv")

In [83]:
metaCSV = pd.read_csv(metadata_csv_path)
metaCSV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51078 entries, 0 to 51077
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   cord_uid                     51078 non-null  object 
 1   sha                          38022 non-null  object 
 2   source_x                     51078 non-null  object 
 3   title                        50920 non-null  object 
 4   doi                          47741 non-null  object 
 5   pmcid                        41082 non-null  object 
 6   pubmed_id                    37861 non-null  float64
 7   license                      51078 non-null  object 
 8   abstract                     42352 non-null  object 
 9   publish_time                 51070 non-null  object 
 10  authors                      48891 non-null  object 
 11  journal                      46368 non-null  object 
 12  Microsoft Academic Paper ID  964 non-null    float64
 13  WHO #Covidence  

In [84]:
metaCSV = metaCSV[['cord_uid', 'sha', 'pmcid', 'title', 'abstract', 'authors', 'journal']]
#metaCSV.head()

In [85]:
#Only keep the items with titles, abstracts, and sha values.
metaCSV.dropna(inplace=True, subset = {'title', 'abstract', 'sha'})


In [59]:
filename = metaCSV.iloc[3,1] + ".json"
filepath = "docs\\CORD-19-research-challenge\\comm_use_subset\\comm_use_subset\\pdf_json\\" + filename
os.path.abspath(filepath)

'C:\\Users\\Nic\\PycharmProjects\\cs5293sp20-project2\\docs\\CORD-19-research-challenge\\comm_use_subset\\comm_use_subset\\pdf_json\\5b68a553a7cbbea13472721cd1ad617d42b40c26.json'

In [198]:
#This block will create a boolean filter to remove all the rows that are not in the comm_use_subset
#But we will use the more comprehensive dataset instead.

#i = 0
#in_subset = []
#for items in metaCSV['sha']:
#    filename = metaCSV.iloc[i,1] + ".json"
#    filepath = "docs\\CORD-19-research-challenge\\comm_use_subset\\comm_use_subset\\pdf_json\\" + filename
#    os.path.abspath(filepath)
#    in_subset.append(os.path.exists(os.path.abspath(filepath)))
#    i = i + 1
#pd.Series(in_subset)
#in_subset
#meta_CSV = metaCSV[in_subset]

In [199]:
loop_count = 0
in_comm_use = []
in_noncomm_use = []
in_custom_license = []
in_biorxiv_medrxiv = []

in_any_subset = []

for items in metaCSV['sha']:
    filename = metaCSV.iloc[loop_count,1] + ".json"
    filepath1 = "docs\\CORD-19-research-challenge\\comm_use_subset\\comm_use_subset\\pdf_json\\" + filename
    filepath2 = "docs\\CORD-19-research-challenge\\noncomm_use_subset\\noncomm_use_subset\\pdf_json\\" + filename
    filepath3 = "docs\\CORD-19-research-challenge\\custom_license\\custom_license\\pdf_json\\" + filename
    filepath4 = "docs\\CORD-19-research-challenge\\biorxiv_medrxiv\\biorxiv_medrxiv\\pdf_json\\" + filename

    in_comm_use.append(os.path.exists(os.path.abspath(filepath1)))
    in_noncomm_use.append(os.path.exists(os.path.abspath(filepath2)))
    in_custom_license.append(os.path.exists(os.path.abspath(filepath3)))
    in_biorxiv_medrxiv.append(os.path.exists(os.path.abspath(filepath4)))
    
    
    if in_comm_use[loop_count] == True or in_noncomm_use[loop_count] == True or in_custom_license[loop_count] == True or in_biorxiv_medrxiv[loop_count] == True:
        in_any_subset.append(True)
    else:
        in_any_subset.append(False)
    loop_count = loop_count + 1
pd.Series(in_any_subset)
#in_any_subset

0        True
1        True
2        True
3        True
4        True
         ... 
33531    True
33532    True
33533    True
33534    True
33535    True
Length: 33536, dtype: bool

In [200]:
meta_CSV = metaCSV[in_any_subset]
meta_CSV.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31646 entries, 0 to 51076
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cord_uid  31646 non-null  object
 1   sha       31646 non-null  object
 2   pmcid     26925 non-null  object
 3   title     31646 non-null  object
 4   abstract  31646 non-null  object
 5   authors   31487 non-null  object
 6   journal   29928 non-null  object
dtypes: object(7)
memory usage: 1.9+ MB


In [100]:
#There are 9,524 items in the comm_use_subset pdf list
#There are 9,148 items in the comm_use_subset pmc list
#There are 2,490 items in the noncomm_use_subset pdf list
#There are 2,217 items in the noncomm_use_subset pmc list
#There are 26,505 items in the custom_license pdf list
#There are 7,802 items in the custom_license pmc list
#There are 1,625 items in the biorxiv_medrxiv pdf list

#That's a total of 40,144 pdf papers (with sha names)
#Using the sha code, we found 31,646 papers.
#So, 8,498 items were not found with the sha code, that's fine.

#We can now take a random sample of these 31,000 papers to generate a much smaller list of 5000.

meta_sample = meta_CSV.sample(n=5000, random_state = 666) #because it's funny
meta_sample


,cord_uid,sha,pmcid,title,abstract,authors,journal
13761,3zturry3,dd63c8a6ff3ae72819472ca9eef5dfb0f13afb5e,PMC7089353,High-level expression of a human β-site APP cl...,Plastid transformation has to date been applie...,"Youm, Jung Won; Jeon, Jae Heung; Kim, Hee; Min...",Transgenic Res
12425,yi2l5bcu,7e4c7d38872c37923d5ba6e8da2c6b01b72f176d,PMC7080100,Review of plasmonic fiber optic biochemical se...,This paper presents a brief overview of the te...,"Caucheteur, Christophe; Guo, Tuan; Albert, Jac...",Anal Bioanal Chem
5169,pd9lt4n2,dd969a7721176db21314930b62e62a37e902dae5,PMC4453572,Heparan Sulfate-Dependent Enhancement of Henip...,"Nipah virus and Hendra virus are emerging, hig...","Mathieu, Cyrille; Dhondt, Kévin P.; Châlons, M...",mBio
5971,r46d5kdu,33310fc6d99fa6bbf3aa6160168f3acba6a736d7,PMC4728157,Roles of the hemagglutinin of influenza A viru...,Seasonal influenza epidemics and influenza pan...,"Jiang, Shibo; Li, Runming; Du, Lanying; Liu, S...",Protein & Cell
50818,e93nu56p,adec4c54ca0073999de9e63c5806585d5ac2c358,PMC7116949,The ventilation of multiple-bed hospital wards...,Abstract Hospital and healthcare facilities ha...,"Yau, Y.H.; Chandrasegaran, D.; Badarudin, A.",Building and Environment
...,...,...,...,...,...,...,...
49765,iwv9lilo,1e90e5ac4f30837179cd3ff9f9f380cd4dcaeca8,PMC7117285,Characterization of turkey coronavirus from tu...,Abstract The present study was to characterize...,"Lin, Tsang L.; Loa, Chien C.; Tsai, Shih C.; W...",Veterinary Microbiology
31306,jngjogrh,67de735e7e3f2e60bfd376d42819b00faa8bd95e,NaN,Disidratazione acuta nel lattante,"Il lattante al di sotto di un anno, e soprattu...","Hubert, P.",EMC - Urgenze
12060,m128nfhn,95cd65dbf317e24c13557dbf46ffbaa4c206f8be,PMC7019308,Isolation and Identification of Porcine Deltac...,Porcine deltacoronavirus (PDCoV) is a porcine ...,"Qian, Shaoju; Jia, Xiangchao; Gao, Zitong; Zha...",Viruses
50715,r1vsvdz0,4a817cc21c19f64d48b5e0a9e062b934eec46955,PMC7126289,A novel method for making human monoclonal ant...,Abstract We have developed a B cell immortaliz...,"Fraussen, J.; Vrolix, K.; Martinez-Martinez, P...",Journal of Autoimmunity


In [210]:
filepaths = []

for i in range (0,len(meta_sample)):
    sha = meta_sample.iloc[i,1] #meta_sample.iloc[i,1]
    filename = sha + ".json"
    
    filepath1 = "docs\\CORD-19-research-challenge\\comm_use_subset\\comm_use_subset\\pdf_json\\" + filename
    filepath2 = "docs\\CORD-19-research-challenge\\noncomm_use_subset\\noncomm_use_subset\\pdf_json\\" + filename
    filepath3 = "docs\\CORD-19-research-challenge\\custom_license\\custom_license\\pdf_json\\" + filename
    filepath4 = "docs\\CORD-19-research-challenge\\biorxiv_medrxiv\\biorxiv_medrxiv\\pdf_json\\" + filename
    
    if os.path.exists(os.path.abspath(filepath1)):
        filepaths.append(filepath1)
    elif os.path.exists(os.path.abspath(filepath2)):
        filepaths.append(filepath2)
    elif os.path.exists(os.path.abspath(filepath3)):
        filepaths.append(filepath3)
    elif os.path.exists(os.path.abspath(filepath4)):
        filepaths.append(filepath4)
    
    
#filepaths[:2]
    
    #globstring = os.path.abspath(os.curdir) + '\\docs\\CORD-19-research-challenge\\**\\pdf_json\\' + filename
    #filepath = glob.glob(globstring, recursive = True)
    #filepaths.append(filepath)

['docs\\CORD-19-research-challenge\\custom_license\\custom_license\\pdf_json\\dd63c8a6ff3ae72819472ca9eef5dfb0f13afb5e.json',
 'docs\\CORD-19-research-challenge\\custom_license\\custom_license\\pdf_json\\7e4c7d38872c37923d5ba6e8da2c6b01b72f176d.json']

In [221]:
#Excellent. Now we need to read in the .json files in a way that allows us to access the text, so we can tokenize it.


full_text_list = []

for i in range (0,len(filepaths)):

    with open(filepaths[i], 'r') as file:
        myjson = json.load(file)
        file.close()


    full_text = ""
    for x in range(0,len(myjson['body_text'])):
        if x == len(myjson['body_text'])-1:
            full_text = full_text + myjson['body_text'][x]["text"]
        else:      
            full_text = full_text + myjson['body_text'][x]["text"] + " "
    
    #Just some helpful progress text since this block takes a few seconds.
    if i % 500 == 0:
        print("text " + str(i) + " processed")
    if i == len(filepaths) - 1:
        print("text " + str(i) + " processed")

    full_text_list.append(full_text)

#full_text_list.head()
#I will actually append this full text to a monster list.





#The goal is to create a dataframe in the end, which we can work with more easily.
#My vision is something like:
#    sha/paper_id     title              abstract_text (list of tokens)          body_text (list of tokens)          TF-IDF vector
#    1200128asfd34    High-level...      'We found that ...'    '[Introduction, Planarians, posess, ability, ...]     [0.5,0.2,0.3,...]



text 0 processed
text 500 processed
text 1000 processed
text 1500 processed
text 2000 processed
text 2500 processed
text 3000 processed
text 3500 processed
text 4000 processed
text 4500 processed
text 4999 processed


In [223]:
full_abstract_list = []

for i in range (0,len(filepaths)):

    with open(filepaths[i], 'r') as file:
        myjson = json.load(file)
        file.close()


    full_abstract_text = ""
    for x in range(0,len(myjson['abstract'])):
        if x == len(myjson['abstract'])-1:
            full_abstract_text = full_abstract_text + myjson['abstract'][x]["text"]
        else:      
            full_abstract_text = full_abstract_text + myjson['abstract'][x]["text"] + " "
    
    if i % 500 == 0:
        print("text " + str(i) + " processed")
    elif i == len(filepaths) - 1:
        print("text " + str(i) + " processed")
        
    full_abstract_list.append(full_abstract_text)

text 0 processed
text 500 processed
text 1000 processed
text 1500 processed
text 2000 processed
text 2500 processed
text 3000 processed
text 3500 processed
text 4000 processed
text 4500 processed
text 4999 processed


In [257]:
#Here is where we can actually start doing some Natural Language Processing.
#Step 1 is always to clean up our text. It is a still kinda a hot mess, lots of extra symbols, tons of words which cloud the
#meaning of the text (called stopwords, "the", "we", "a", etc. stuff like that)


#nltk.download('stopwords') need this for the first run.
stop_words = nltk.corpus.stopwords.words('english')
extraStopWords = ['et','al', 'al.', '(', ')', ',', '.', ':', ';', '%', '#', '@', '&', '!', '?', '[', ']']
stop_words.extend(extraStopWords)

tokenized_text = []

def normalizeDoc(txt):
    for i in range(0,len(txt)):
        myText = txt[i]
        myText = re.sub(r'[^a-zA-Z0-9\s]', '', myText, re.I|re.A)
        myText = myText.lower()
        myText = myText.strip()
        tokens = nltk.word_tokenize(myText)
        clean_tokens = [t for t in tokens if t not in stop_words]
        tokenized_text.append(clean_tokens)
        if i % 100 == 0:
            print("text " + str(i) + " tokenized ("+ str(i/len(txt)*100) + "% complete)")
        elif i == len(txt) - 1:
            print("text " + str(i) + " tokenized (100% complete)")
            
normalizeDoc(full_text_list)
len(tokenized_text)


text 0 tokenized (0.0% complete)
text 100 tokenized (2.0% complete)
text 200 tokenized (4.0% complete)
text 300 tokenized (6.0% complete)
text 400 tokenized (8.0% complete)
text 500 tokenized (10.0% complete)
text 600 tokenized (12.0% complete)
text 700 tokenized (14.000000000000002% complete)
text 800 tokenized (16.0% complete)
text 900 tokenized (18.0% complete)
text 1000 tokenized (20.0% complete)
text 1100 tokenized (22.0% complete)
text 1200 tokenized (24.0% complete)
text 1300 tokenized (26.0% complete)
text 1400 tokenized (28.000000000000004% complete)
text 1500 tokenized (30.0% complete)
text 1600 tokenized (32.0% complete)
text 1700 tokenized (34.0% complete)
text 1800 tokenized (36.0% complete)
text 1900 tokenized (38.0% complete)
text 2000 tokenized (40.0% complete)
text 2100 tokenized (42.0% complete)
text 2200 tokenized (44.0% complete)
text 2300 tokenized (46.0% complete)
text 2400 tokenized (48.0% complete)
text 2500 tokenized (50.0% complete)
text 2600 tokenized (52.0% 

5000

In [258]:
tokenized_text[3:4]

[['influenza',
  'continues',
  'pose',
  'serious',
  'threats',
  'public',
  'health',
  'worldwide',
  'since',
  'seasonal',
  'influenza',
  'epidemics',
  'affect',
  '15',
  'population',
  'result',
  '500000',
  'deaths',
  'worldwide',
  'year',
  'httpwwwwhointcsrdisease',
  'influenzaresearchagendadocumentpdf',
  'influenza',
  'pandemics',
  'like',
  '1918',
  'flu',
  'pandemic',
  'spanish',
  'flu',
  'may',
  'lead',
  'millions',
  'deaths',
  'taubenberger',
  '2006',
  '2009',
  'new',
  'influenza',
  'pandemic',
  'caused',
  'novel',
  'swineorigin',
  'influenza',
  'virus',
  'soiv',
  'h1n1',
  'resulted',
  'millions',
  'infections',
  '213',
  'countries',
  'overseas',
  'territories',
  'communities',
  'httpwwwwhointcsrdon',
  '20100312enindexhtml',
  'miller',
  '2009',
  'smith',
  '2009',
  'itoh',
  '2009',
  'rapid',
  'spread',
  'soiv',
  'humans',
  'worldwide',
  'continuous',
  'mutations',
  'soiv',
  'proteins',
  'pan',
  'jiang',
  '2009'